# Preprocessing        

## merge $\rightarrow$ demean $\rightarrow$ detrend $\rightarrow$ taper

In [ ]:
# merge->demean->detrend->taper
import obspy
from obspy import read
import os
from obspy.io.sac import attach_paz
from obspy.signal.invsim import corn_freq_2_paz
from obspy.signal import PPSD
import glob
# Define the directory containing your seismic data files (EPZ).
data_directory = '/raid1/SM_data/archive/2023/TW/SM01/EPZ.D/'
output_dir = '/raid1/SM_data/archive/2023/TW/SM01_ppreprocessing'
os.makedirs(output_dir, exist_ok=True)
# exist_ok, when dir is exist that won't exist error.

# Use glob to find all files with a specific extension (e.g., .mseed) in the directory.
data_files = glob.glob(os.path.join(data_directory, '*EPZ*'))

# Loop through each data file and process them.
for data_file in data_files:
    # Read the seismic data file into a Stream object.
    st = read(data_file)

    # Merge traces within the Stream if there is more than one trace.
    if len(st) > 1:
        st = st.merge(method=1, fill_value=0)
        # method 1: more flexible, but for our purpose there's no difference between using method 0 or 1.
        # filled = 0, that's a default, we don't want to interpolate the value.
    # Preprocess the Stream.
    st.detrend('demean') # mean of data is subtracted.
    st.detrend('linear') # using least square to find a fitting line and subtract it.
    st.taper(type='hann', max_percentage=0.05) # to make 5 % waveform in both endpoint be flattened by hanning window.

    # Define the output file path and save the preprocessed Stream.
    output_file = os.path.join(output_dir, os.path.basename(data_file))
    st.write(output_file, format='SAC')

print("Preprocessing and saving completed.")

## instrument response     

### attach_paz()    $\rightarrow$ no need to use, just for understanding    
Examples from [here!](<https://docs.obspy.org/packages/autogen/obspy.io.sac.sacpz.attach_paz.html#obspy.io.sac.sacpz.attach_paz>)

In [3]:
from obspy import Trace
from obspy.io.sac import attach_paz
import io
tr = Trace()
f = io.StringIO("""ZEROS 3 
-5.032 0.0
POLES 6
-0.02365 0.02365
-0.02365 -0.02365
-39.3011 0.
-7.74904 0.
-53.5979 21.7494
-53.5979 -21.7494
CONSTANT 2.16e18""")
# using triple quote to create multi-line string
attach_paz(tr, f,torad=True) 
# actually I don't understand the torad, it's not important enough to know.
# the importance of using attach_paz just to attach the AttribDict onto the stats.
tr.stats
# using .stats to confirm 

         network: 
         station: 
        location: 
         channel: 
       starttime: 1970-01-01T00:00:00.000000Z
         endtime: 1970-01-01T00:00:00.000000Z
   sampling_rate: 1.0
           delta: 1.0
            npts: 0
           calib: 1.0
             paz: AttribDict({'seismometer_gain': 1.0, 'digitizer_gain': 1.0, 'poles': [(-0.14859733251479723+0.14859733251479723j), (-0.14859733251479723-0.14859733251479723j), (-246.9360940759956+0j), (-48.6886542727469+0j), (-336.76553777568074+136.6555105199717j), (-336.76553777568074-136.6555105199717j)], 'zeros': [(-31.61698846572768+0j), 0j, 0j], 'sensitivity': 1.0, 'gain': 2.16e+18})

### corn_freq_2_paz()    
the function below is provided by the [ObsPy.org](<https://docs.obspy.org/_modules/obspy/signal/invsim.html#corn_freq_2_paz>)

In [ ]:
import math
import numpy as np
def corn_freq_2_paz(fc, damp=0.707):
    """
    Convert corner frequency and damping to poles and zeros.

    2 zeros at position (0j, 0j) are given as output  (m/s).

    :param fc: Corner frequency
    :param damping: Corner frequency
    :return: Dictionary containing poles, zeros and gain
    """
    poles = [-(damp + math.sqrt(1 - damp ** 2) * 1j) * 2 * np.pi * fc,
             -(damp - math.sqrt(1 - damp ** 2) * 1j) * 2 * np.pi * fc]
    return {'poles': poles, 'zeros': [0j, 0j], 'gain': 1, 'sensitivity': 1.0}

# As the function shows, only the poles will be converted, other keys are fixed.

### simulate()     
the most important part, giving a new instrument response parameter and conducting the simulation.

In [ ]:
from obspy import read
from obspy.signal.invsim import corn_freq_2_paz
st = read()
paz_sts2 = {'poles': [-0.037004+0.037016j, -0.037004-0.037016j,
                      -251.33+0j,
                      -131.04-467.29j, -131.04+467.29j],
            'zeros': [0j, 0j],
            'gain': 60077000.0,
            'sensitivity': 2516778400.0}
paz_1hz = corn_freq_2_paz(1.0, damp=0.707)
st.simulate(paz_remove=paz_sts2, paz_simulate=paz_1hz) # The instrment response is removed by given values, and simulate in a value defined by corn_freq_2_paz

st.plot()  

# Problem solver
1. I ran and wrote the data into my directory throught the loop iterating, however, it stopped at a point and my data was truncated. How can I avoid to ran again the data I already ran, in other words, can I ignore the data I already ran just like the os.makesdir(exist_ok=True) to pass it?
2.  I set up a loop to acquire the data from each station in a specific threshold, for example, only retrive the data that end with 1 to 10(file_1.txt,file_2.txt,...file_10.txt). My setting is ok that I can sieve the data, however, not every station have all 1 to 10 data, the empty list was generated when I lack of the data, and I want to ignore that, if I only have file_1.txt to file_4.txt, it can automatically stop to search further data, maybe I can use try to solve it?    
okay I just fix it by using the try and except.
3. Learning about how to use logging to document the error and the progress.   

In [1]:
from obspy import read
path = '/raid1/SM_data/archive/2023/TW/remove_resp/SM01/TW.SM01.00.EPZ.D.2023.222'
st = read(path)
st[0].stats

         network: TW
         station: SM01
        location: 00
         channel: EPZ
       starttime: 2023-08-10T00:00:00.720000Z
         endtime: 2023-08-11T00:00:00.530000Z
   sampling_rate: 100.0
           delta: 0.01
            npts: 8639982
           calib: 1.0
         _format: SAC
             sac: AttribDict({'delta': 0.01, 'depmin': -3.0933045e-08, 'depmax': 3.2719115e-08, 'scale': 1.0, 'b': 0.0, 'e': 86399.805, 'depmen': 2.628723e-13, 'nzyear': 2023, 'nzjday': 222, 'nzhour': 0, 'nzmin': 0, 'nzsec': 0, 'nzmsec': 720, 'nvhdr': 6, 'npts': 8639982, 'iftype': 1, 'iztype': 9, 'leven': 1, 'lpspol': 1, 'lovrok': 1, 'lcalda': 0, 'kstnm': 'SM01', 'khole': '00', 'kcmpnm': 'EPZ', 'knetwk': 'TW'})